In [2]:
import xml.etree.ElementTree as ET
import os
import dill
import pandas as pd
from datetime import datetime
import operator
import re
import bokeh

from sklearn import base
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import FeatureUnion, Pipeline

os.chdir('/Users/longy/Documents/TDI/capstone')
df = dill.load(open('df.pkd', 'r'))

In [3]:
tag_count = dict()
for tag in df.q_tag:
    li = tag.split('><')
    for j in li:
        j = j.replace('<', '')
        j = j.replace('>', '')
        if j not in tag_count:
            tag_count[j] = 1
        else:
            tag_count[j] += 1
sortedTags = sorted(tag_count.items(), key = operator.itemgetter(1), reverse = True)
selectedTags = {k:v for k,v in sortedTags if v>=len(df)*0.01} #use 1% as threshold to create matrix for tags

In [4]:
selectedTags #tags that have at least 1% coverage in df

{'.net': 10512,
 'ajax': 1185,
 'algorithm': 1108,
 'asp.net': 7212,
 'asp.net-mvc': 1959,
 'c': 2207,
 'c#': 15092,
 'c++': 5771,
 'css': 1873,
 'database': 2131,
 'html': 2724,
 'iphone': 1681,
 'java': 7855,
 'javascript': 5307,
 'jquery': 2319,
 'linq': 1177,
 'linux': 1517,
 'multithreading': 1176,
 'mysql': 2279,
 'performance': 1524,
 'php': 4458,
 'python': 4005,
 'regex': 1325,
 'ruby': 1697,
 'ruby-on-rails': 1660,
 'security': 1098,
 'sql': 3857,
 'sql-server': 3658,
 'svn': 1236,
 'unit-testing': 1109,
 'vb.net': 1799,
 'visual-studio': 2081,
 'visual-studio-2008': 1424,
 'web-services': 1170,
 'windows': 2554,
 'winforms': 1706,
 'wpf': 1867,
 'xml': 1737}

In [5]:
class DictEncoder(base.BaseEstimator, base.TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        result = []
        for lst in X:
            arr = lst.split('><')
            d = dict()
            for i in arr:
                i = i.replace('<', '')
                i = i.replace('>', '')
                if i in selectedTags:
                    d[i] = 1
            result.append(d)
        return result
    
de = DictEncoder()
dv = DictVectorizer(sparse = False)
tag_dict = de.fit_transform(df.q_tag)
tag_mat = dv.fit_transform(tag_dict)

In [6]:
tag_pd = pd.DataFrame(tag_mat, columns = dv.get_feature_names())
df_tag = pd.concat([df, tag_pd], axis = 1)

In [23]:
tag_summary = []
for name in dv.get_feature_names():
    summary = df_tag.groupby(name)['fa_time'].describe()
    summary['name'] = name
    tag_summary.append(summary)
tag_summary = pd.concat(tag_summary)

In [25]:
dill.dump(tag_summary, open('tag_summary.pkd', 'w'))
dill.dump(df_tag, open('df_tag.pkd', 'w'))
# df_tag.to_csv('viz/df_tag.csv', encoding='utf-8')